In [3]:
# !pip install pyautogui
# !pip install psutil
# !pip install opencv-python
# !pip install mss
# !pip install pyttsx3
# !pip install chime

In [4]:
import pyautogui
import subprocess
from mss import mss
from PIL import Image
import time
import pyttsx3
import chime

engine = pyttsx3.init()

activated_speech = True

def say(text_to_speak):
    # List available voices
    if activated_speech:
        voices = engine.getProperty('voices')

        # Set a specific voice (replace 1 with the index of the desired voice)
        engine.setProperty('voice', voices[1].id)

        # Set the rate of speech
        engine.setProperty('rate', 250)

        # Set the volume (0.0 to 1.0)
        engine.setProperty('volume', 1.5)
        engine.say(text_to_speak)
        engine.runAndWait()

def locate_and_click(icon_filename):
    # take_screenshot("screenshot.png")
    location = pyautogui.locateOnScreen(icon_filename, confidence=0.8)
    if location:
        print(f"The icon is located at {location}")
        # Get the center of the located image, as `click()` clicks at the center of the given region
        center_location = pyautogui.center(location)
        pyautogui.click(center_location)
        print("Clicked on the icon.")
        return True
    else:
        print("The icon could not be found.")
        return False


def run_hearthstone():
    path_to_launcher = "C:\\Program Files (x86)\\Hearthstone\\Hearthstone Beta Launcher.exe"
    subprocess.run(path_to_launcher)


# Locate the Battle.net icon on the screenshot and click it
def wait(seconds=60):
    time.sleep(seconds)

def run_all(l):
    odd, even = [], []
    for i, x in enumerate(l):
        if i % 2 == 0:
            even.append(x)
        else:
            odd.append(x)
    if len(odd) < len(even):
        odd.append(1)
    for i in range(len(odd)):
        print('finding : ', even[i])
        good = locate_and_click(even[i])
        name = even[i]
        if not good:
            if  name == '8_fight.png':
                locate_and_click('7_played.png')

            if name == '0_battlenet_icon.png':
                # run the client
                run_hearthstone()
                # retry 
                wait(20)
                run_all(['0_battlenet_icon.png', 3])

        wait(odd[i])

In [5]:
# check condition -> action 
def detection(cond_to_act):
    while True:
        found = False
        for cond in cond_to_act:
            if cond.check_condition():
                if cond.click_action():
                    found = True
                    break
        if not found:
            say("No condition found")
            # click somewhere to continue
            # x=859, y=8)
            pyautogui.click(859, 0)
        wait(1)

def action():
    pass 

In [6]:
class ConditionToAction:
    def __init__(self, name, wait_time_between=0, wait_time=3):
        self.name = name
        self.condition = []
        self.action = []
        self.wait_time = wait_time
        self.wait_time_between = wait_time_between

    def add_condition_image(self, condition):
        self.condition.append(condition)
    
    def check_condition(self):
        if self.locate():
            print(f"** Dectected {self.name} ")
            say(f"Detected {self.name} ")
            return True
    
    def general_locate(self, image):
        local = pyautogui.locateOnScreen(image, confidence=0.7)
        if local:
            return True, pyautogui.center(local)
        else:
            return False, None

    def general_click(self, image):
        found, location = self.general_locate(image)
        if found:
            pyautogui.click(location)
            return True
        else:
            print(f"** {image} not found")
            return False
    
    def locate(self):
        # print("Try to locate " + self.name + " on the screen.")
        condifence = 0.7
        if self.name == 'reward':
            condifence = 0.5
        if self.name == 'fight':
            condifence = 0.6
        if self.name == 'pick treasure':
            condifence = 0.6

        for condition in self.condition:
            location = pyautogui.locateOnScreen(condition, confidence=condifence)
            if location:
                print(f"The icon is located at {location}")
                # Get the center of the located image, as `click()` clicks at the center of the given region
                # center_location = pyautogui.center(location)
                # pyautogui.click(center_location)
                # print("Clicked on the icon.")
                return True
            else:
                print("The icon could not be found.")
        return False

    def add_action_image(self, *args):
        # if args == ():
            # self.action.append([name_action, action])
        # else:
        self.action.append(args)
        print('- Added:', self.action[-1])
    
    def check_play(self, click=False):
        found, location = self.general_locate('encounter/1_play.png')
        if found:
            print('play is available')
            if click:
                # click play
                pyautogui.click(location)
            return True
        return False
    
    def encounter(self):
        # check if play is available
        if self.check_play(click=True):
            return 
        else:
            # say('finding boss')
            possible_boss =['encounter/1_boss1.png', 'encounter/2_boss2.png', 'encounter/3_boss3.png', 'encounter/4_boss4.png', 'encounter/5_boss5.png']
            found_boss = False
            for b in possible_boss:
                found_boss = self.general_click(b)
                wait(1)
                if self.check_play(click=True):
                    return
            # if not found_boss:
            print('Boss not found')
            # random click, in the middle of the screen
            import random
            for i in range(5):
                pyautogui.click(random.randint(500, 1000), random.randint(300, 700))
            return

    def locate_enemies(self):
        enemies = [
           'fight/enemy1.png',
           'fight/enemy2.png',
           'fight/enemy3.png',
           'fight/enemy4.png',
           'fight/enemy5.png',
           'fight/enemy6.png',
           'fight/enemy7.png',
            'fight/enemy8.png',
            'fight/enemy9.png',
            'fight/enemy10.png',
            'fight/enemy11.png',
        ]
        pos = []
        for enemy in enemies:
            found, location = self.general_locate(enemy)
            if found:
                pos.append(location)
        if len(pos) < 0:
            say('no enemy found')
        return pos

    def click_action(self):
        if self.name == 'fight':
            def drag(end_x, end_y, duration=1.0):
                start_x, start_y = 742, 520
                pyautogui.moveTo(start_x, start_y)
                duration = 0.5
                pyautogui.dragTo(end_x, end_y, duration, button='left')
                wait(3)

            # say('click first troop')
            self.general_click('fight/3_first_troop.png')

            enemies_pos = self.locate_enemies()
            if enemies_pos == []:
                enemies_pos = [(965, 332), (965, 332), (965, 332)]
            elif len(enemies_pos) < 3:
                enemies_pos.append(enemies_pos[0])
                if len(enemies_pos) < 3:
                    enemies_pos.append(enemies_pos[0])

            for pos in enemies_pos:
                drag(pos[0], pos[1])
            wait(1)
            # click right cursor
            pyautogui.click(1152, 468, button='right')

        
        if self.name == 'pick treasure':
            pyautogui.click(1152, 468)
            wait(2)
        if self.name == 'encounter':
            self.encounter()
            wait(self.wait_time)
            return
        
        if self.name == 'reward':
            # click
            pyautogui.click(677, 809)
            wait(1)
            pyautogui.click(1006, 309)
            wait(1)
            pyautogui.click(1333, 795)
            wait(2)

        for action_chain in self.action:
            for i in range(0, len(action_chain), 2):
                name, action = action_chain[i], action_chain[i+1]
                print('  - ', name)
                confidence = 0.8
                if name == 'reward':
                    confidence = 0.5
                if name == 'fight':
                    confidence = 0.6
                location = pyautogui.locateOnScreen(action, confidence=confidence)
                if location:
                    print(f"    #The action is located at {location}")
                    if name != '':
                        say(f"{name}")
                    center_location = pyautogui.center(location)
                    pyautogui.click(center_location)
                    # return True
                else:
                    print("     #The action could not be found.")
                wait(self.wait_time_between)
        
        wait(self.wait_time)
        if self.name == 'game start':
            # keep pc awake for 29 minutes, click every 5 minutes
            for i in range(5):
                # wait(5.5 * 60)
                wait(60)
                pyautogui.click(876, 0)
        

In [7]:
desktop_condition = ConditionToAction('desktop', wait_time_between=2)
desktop_condition.add_condition_image('desktop/desktop.png')
desktop_condition.add_action_image('open Hearth Stone', 'desktop/0_hearthstone_icon.png')
desktop_condition.add_action_image(
    'open battle net', 'desktop/0_battlenet_icon.png',
    '', 'desktop/2_games.png',
    '', 'desktop/3_hearthstone.png',
    'click play', 'desktop/1_play.png'
)

main_menu_condition = ConditionToAction('main menu')
main_menu_condition.add_condition_image('main_menu/0_main_menu.png')
main_menu_condition.add_action_image(
    'click modes', 'main_menu/1_mode.png',
)
modes_condition = ConditionToAction('modes')
modes_condition.add_condition_image('modes/0_choose_a_mode.png')
modes_condition.add_action_image(
    'click mercenaries', 'main_menu/2_mer.png',
    'click choose', 'main_menu/3_choose.png',
)
mercenaries_condition = ConditionToAction('mercenaries')
mercenaries_condition.add_condition_image('mercenaries/0_mercenaries.png')
mercenaries_condition.add_action_image(
    'click play', 'mercenaries/1_camp.png',
)
travel_point_condition = ConditionToAction('travel point')
travel_point_condition.add_condition_image('travel_point/0_travel_point.png')
travel_point_condition.add_action_image(
    'choose travel point', 'travel_point/1_choose.png',
)
bounties_condition = ConditionToAction('bounties')
bounties_condition.add_condition_image('bounties/0_bounties.png')
bounties_condition.add_action_image(
    'choose boss', 'bounties/1_boss.png',
    'click choose', 'bounties/2_choose.png',
)
party_condition = ConditionToAction('choose a party', wait_time_between=2)
party_condition.add_condition_image('choose_a_party/0_choose_a_party.png')
party_condition.add_action_image(
    'click choose', 'choose_a_party/1_choose.png',
    'lock in', 'choose_a_party/2_lock_in.png',
)
# first_boss_condition = ConditionToAction('first boss', wait_time=5)
# first_boss_condition.add_condition_image('boss1/0_boss_1.png')
# first_boss_condition.add_action_image(
#     'click play', 'boss1/1_play.png',
# )
fight_condition = ConditionToAction('fight', wait_time=15)
fight_condition.add_condition_image('fight/0_fight.png')
fight_condition.add_condition_image('fight/0_fight2.png')
fight_condition.add_condition_image('fight/4_condition.png')
fight_condition.add_action_image(
    'click fight', 'fight/0_fight.png',
    'click ready', 'fight/5_ready.png',
    'click out', 'fight/2_click_out.png',
)
pick_treasure_condition = ConditionToAction('pick treasure', wait_time_between=2)
pick_treasure_condition.add_condition_image('pick_treasure/0_pick_treasure.png')
pick_treasure_condition.add_action_image(
    'take', 'pick_treasure/1_take.png',
)
encounter_condition = ConditionToAction('encounter', wait_time=12)
encounter_condition.add_condition_image('encounter/0_encounter.png')
encounter_condition.add_action_image(
    'click play', 'encounter/1_play.png',
)
game_start_condition = ConditionToAction('game start', wait_time=15, wait_time_between=2)
game_start_condition.add_condition_image('game_start/0_played.png')
game_start_condition.add_action_image(
    'click play', 'game_start/0_played.png',
    'click out', 'fight/2_click_out.png',
    'quit', 'quit/0_quit.png',
    'quit', 'quit/1_quit.png',
)
click_to_continue_condition = ConditionToAction('click to continue', wait_time_between=1)
click_to_continue_condition.add_condition_image('click_to_continue/0_click_to_continue.png')
click_to_continue_condition.add_action_image(
    'click to continue', 'click_to_continue/0_click_to_continue.png',
    'click to continue', 'click_to_continue/0_click_to_continue.png',
    'click to continue', 'click_to_continue/0_click_to_continue.png',
    'click to continue', 'click_to_continue/0_click_to_continue.png',
)
reward_condition = ConditionToAction('reward', wait_time_between=2)
reward_condition.add_condition_image('reward/0_reward.png')
reward_condition.add_condition_image('reward/1_reward.png')
reward_condition.add_condition_image('reward/2_reward.png')
reward_condition.add_condition_image('reward/5_reward.png')
reward_condition.add_action_image(
    'click reward', 'reward/0_reward.png',
    'click done', 'reward/4_done.png',
)
done_condition = ConditionToAction('done', wait_time_between=2)
done_condition.add_condition_image('reward/4_done.png')
done_condition.add_action_image(
    'click done', 'reward/4_done.png',
)
bounties_completed_condition = ConditionToAction('bounties completed', wait_time_between=2)
bounties_completed_condition.add_condition_image('bounties/3_bounties_completed.png')
bounties_completed_condition.add_action_image(
    'click ok', 'ok.png',
)

# x=1364, y=881


conditions =[
    desktop_condition,
    main_menu_condition,
    modes_condition,
    mercenaries_condition,
    travel_point_condition, 
    bounties_condition,
    party_condition,
    # first_boss_condition,
    fight_condition,
    pick_treasure_condition,
    encounter_condition,
    game_start_condition,
    click_to_continue_condition,
    reward_condition,
    done_condition,
    bounties_completed_condition,
]
# 3-4 click

- Added: ('open Hearth Stone', 'desktop/0_hearthstone_icon.png')
- Added: ('open battle net', 'desktop/0_battlenet_icon.png', '', 'desktop/2_games.png', '', 'desktop/3_hearthstone.png', 'click play', 'desktop/1_play.png')
- Added: ('click modes', 'main_menu/1_mode.png')
- Added: ('click mercenaries', 'main_menu/2_mer.png', 'click choose', 'main_menu/3_choose.png')
- Added: ('click play', 'mercenaries/1_camp.png')
- Added: ('choose travel point', 'travel_point/1_choose.png')
- Added: ('choose boss', 'bounties/1_boss.png', 'click choose', 'bounties/2_choose.png')
- Added: ('click choose', 'choose_a_party/1_choose.png', 'lock in', 'choose_a_party/2_lock_in.png')
- Added: ('click fight', 'fight/0_fight.png', 'click ready', 'fight/5_ready.png', 'click out', 'fight/2_click_out.png')
- Added: ('take', 'pick_treasure/1_take.png')
- Added: ('click play', 'encounter/1_play.png')
- Added: ('click play', 'game_start/0_played.png', 'click out', 'fight/2_click_out.png', 'quit', 'quit/0_quit.png', 'q

In [8]:
while True:
    try:
        detection(conditions)
    except Exception as e:
        # print(e)
        # print('error')
        chime.error()
        raise(Exception)
        pass
    wait(10)

The icon is located at Box(left=315, top=1140, width=345, height=57)
** Dectected desktop 
  -  open Hearth Stone
    #The action is located at Box(left=1428, top=1142, width=44, height=46)
  -  open battle net
     #The action could not be found.
  -  
     #The action could not be found.
  -  
     #The action could not be found.
  -  click play
     #The action could not be found.
The icon is located at Box(left=342, top=28, width=1230, height=908)
** Dectected main menu 
  -  click modes
    #The action is located at Box(left=812, top=597, width=297, height=50)
The icon is located at Box(left=618, top=0, width=586, height=111)
** Dectected modes 
  -  click mercenaries
    #The action is located at Box(left=1257, top=170, width=274, height=279)
  -  click choose
    #The action is located at Box(left=1272, top=693, width=284, height=269)
The icon is located at Box(left=730, top=1, width=464, height=88)
** Dectected mercenaries 
  -  click play
    #The action is located at Box(left

  -  open battle net
    #The action is located at Box(left=1429, top=1142, width=43, height=45)


In [ ]:
import pyautogui
import time

while True:
    current_position = pyautogui.position()
    print(current_position)
    time.sleep(1)
# x=662, y=871
# x=1364, y=881

Point(x=484, y=321)
Point(x=872, y=0)
Point(x=845, y=4)
Point(x=859, y=8)
Point(x=884, y=6)
Point(x=805, y=68)


KeyboardInterrupt: 

In [ ]:
# run_all(
#     [
#         '0_battlenet_icon.png',
#         1,
#         # '1_play.png',
#         # 30,
#         # '2_mode.png',
#         # 3,
#         # '3_mer.png',
#         # 1,
#         # '4_choose.png',
#         # 1,
#         '5_camp.png',
#         1
#         '6_play_round.png',
#         2,
#         '7_played.png',
#         6,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#     ]
# )

In [ ]:
# # for test
# locate_and_click('0_hearthstone_icon.png')
# wait(3)
# # locate_and_click('2_mode.png')
# # wait(10)
# # locate_and_click('3_mer.png')
# # wait(3)
# # locate_and_click('4_choose.png')
# wait(3)
# locate_and_click('5_camp.png')
# wait(4)
# locate_and_click('play.png')
# run_all(
#     [
#         '0_hearthstone_icon.png',
#         # 3,
#         # '2_mode.png',
#         # 10,
#         # '3_mer.png',
#         # 3,
#         # '4_choose.png',
#         # 2,
#         # '5_camp.png',
#         # 1,
#         '6_play_round.png',
#         2,
#         '7_played.png',
#         6,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#         '8_fight.png',
#         7,
#     ]
# )
# pyautogui.click(location)